Copyright (c) 2023 by the LEL-A team

This software is distributed under the terms of the MIT license
which is available at https://opensource.org/licenses/MIT

In [1]:
import random
import uuid
import pandas as pd
import gzip
import xmltodict

In [2]:
!mkdir -p src_data

In [3]:
# -nc option: do not download if available
!wget -nc -O ./src_data/OPUS_Tatoeba_v2022_03_03_tmx_de_en.tmx.gz https://object.pouta.csc.fi/OPUS-Tatoeba/v2022-03-03/tmx/de-en.tmx.gz

File ‘./src_data/OPUS_Tatoeba_v2022_03_03_tmx_de_en.tmx.gz’ already there; not retrieving.


In [4]:
DATASET_NAME_AND_VERSION = "Instruct_OPUS_Tatoeba_v2022_03_03_de_en_v1"

In [5]:
random.seed(42)

In [6]:
with gzip.open(
    "./src_data/OPUS_Tatoeba_v2022_03_03_tmx_de_en.tmx.gz"
) as open_file:
    tmx_data = xmltodict.parse(open_file)

In [7]:
# unpack tmx_data to lists

de_texts = []
en_texts = []

for tu in tmx_data["tmx"]["body"]["tu"]:
    de_text = None
    en_text = None
    for tuv in tu["tuv"]:
        lang = tuv["@xml:lang"]
        text = tuv["seg"]
        assert lang in ["de", "en"]
        text = str(text).strip()
        assert len(text) > 0, f"{len(text)}, {text}"
        if lang == "de":
            if de_text is None:
                de_text = text
            else:
                assert False, "duplicate de entry should not happen"
        elif lang == "en":
            if en_text is None:
                en_text = text
            else:
                assert False, "duplicate en entry should not happen"
        else:
            assert False, "unknown language"
    assert de_text is not None
    assert en_text is not None
    
    assert isinstance(de_text, str)
    assert isinstance(en_text, str)
    
    assert len(de_text) > 2, de_text
    assert len(en_text) > 2, en_text
    
    de_texts.append(de_text)
    en_texts.append(en_text)
    assert len(de_texts) == len(en_texts)

In [8]:
def print_pair_example(index):
    print(en_texts[index])    
    print(de_texts[index])

In [9]:
print_pair_example(10)

Muiriel has turned twenty.
Muiriel ist jetzt 20.


In [10]:
print_pair_example(-1)

The wine is in the market.
Der Wein ist auf dem Markt.


In [11]:
def create_synonyms():
    """Select synonyms from synonyms dict."""
    return {k: random.choice(v) for (k, v) in synonyms.items()}

In [12]:
synonyms = {
    "text_de": ["Text", "Absatz", "Textbereich", "Abschnitt"],
    "text_en": ["text", "paragraph", "text area", "section"],    
    "exakt_de": ["exakt", "genau", "präzise", "akkurat"],
    "exakt_en": ["exactly", "accurately", "precisely"],
    "ist_de": ["ist", "lautet", "heißt"],    
    "ist_en": ["is", "reads", "is called"],
}

In [13]:
input_templates_to_en = [
# de
"""\
Bitte übersetze folgenden {text_de} ins Englische. Antworte nur {exakt_de} mit der Übersetzung.
Der {text_de} {ist_de}: {input}""",

"""\
Gegeben ist folgender {text_de}: {input}
Übersetze den {text_de} ins Englische und antworte nur {exakt_de} mit der Übersetzung.""",

"""\
Gegeben ist der {text_de}: {input}
Übertrage den {text_de} in englische Sprache und antworte nur {exakt_de} mit der Übersetzung.""",    
    
"""\
Übersetz den folgenden {text_de} ins Englische und erzeuge nur {exakt_de} die Übersetzung.
{input}""",

"""\
Sag mir, was folgender {text_de} auf Englisch heißt. Gebe mir {exakt_de} nur die Übersetzung.
{input}""",    
    
# en
"""\
Please translate the following {text_en} into English. Answer only {exakt_en} with the translation.
The {text_en} {ist_en}: {input}""",

"""\
Given is the following {text_en}: {input}
Translate the {text_en} into English and answer only {exakt_en} with the translation.""",

"""\
Given is the following {text_en}: {input}
Transfer the {text_en} into English and answer only {exakt_en} with the translation.""",    

"""\
Translate the following {text_en} into English and create only the exact translation.
{input}""",  
    
"""\
Tell me what the following {text_en} is called in English. Give me {exakt_en} only the translation.
{input}""",        
]

In [14]:
input_templates_to_de = [
# de
"""\
Bitte übersetze folgenden {text_de} ins Deutsche. Antworte nur {exakt_de} mit der Übersetzung.
Der {text_de} {ist_de}: {input}""",

"""\
Gegeben ist folgender {text_de}: {input}
Übersetze den {text_de} ins Deutsche und antworte nur {exakt_de} mit der Übersetzung.""",

"""\
Gegeben ist der {text_de}: {input}
Übertrage den {text_de} in deutsche Sprache und antworte nur {exakt_de} mit der Übersetzung.""",    
    
"""\
Übersetz den folgenden {text_de} ins Deutsche und erzeuge nur {exakt_de} die Übersetzung.
{input}""",

"""\
Sag mir, was folgender {text_de} auf Deutsch heißt. Gebe mir {exakt_de} nur die Übersetzung.
{input}""",    
    
# en
"""\
Please translate the following {text_en} into German. Answer only {exakt_en} with the translation.
The {text_en} {ist_en}: {input}""",

"""\
Given is the following {text_en}: {input}
Translate the {text_en} into German and answer only {exakt_en} with the translation.""",

"""\
Given is the following {text_en}: {input}
Transfer the {text_en} into German and answer only {exakt_en} with the translation.""",    

"""\
Translate the following {text_en} into German and create only the exact translation.
{input}""",  
    
"""\
Tell me what the following {text_en} is called in German. Give me {exakt_en} only the translation.
{input}""",        
]

In [15]:
input_texts = []
output_texts = []
src_langs = []
target_langs = []

for de_text, en_text in zip(de_texts, en_texts):
    s = create_synonyms()

    to_en_input = random.choice(input_templates_to_en)
    to_en_input = to_en_input.format(**s, input=de_text)
    if "{" in to_en_input:
        print(to_en_input)
    input_texts.append(to_en_input)
    output_texts.append(en_text)
    src_langs.append("de")
    target_langs.append("en")
    
    to_de_input = random.choice(input_templates_to_de)
    to_de_input = to_de_input.format(**s, input=en_text)
    if ("{" in to_de_input) or ("}" in to_de_input):
        print(to_de_input)
    input_texts.append(to_de_input)
    output_texts.append(de_text)
    src_langs.append("en")
    target_langs.append("de")    

assert len(input_texts) == len(output_texts) == len(src_langs) == len(target_langs)

# Explore Data

In [16]:
def print_example(index):
    print(input_texts[index]) 
    print()
    print(output_texts[index])
    print()    
    print(f"from {src_langs[index]} to {target_langs[index]}")    

In [17]:
print_example(0)

Gegeben ist folgender Text: Lass uns etwas versuchen!
Übersetze den Text ins Englische und antworte nur präzise mit der Übersetzung.

Let's try something.

from de to en


In [18]:
print_example(1)

Translate the following text into German and create only the exact translation.
Let's try something.

Lass uns etwas versuchen!

from en to de


In [19]:
print_example(10)

Tell me what the following text is called in English. Give me exactly only the translation.
Was ist das?

What's this?

from de to en


In [20]:
print_example(11)

Please translate the following text into German. Answer only exactly with the translation.
The text reads: What's this?

Was ist das?

from en to de


In [21]:
print_example(-2)

Sag mir, was folgender Text auf Englisch heißt. Gebe mir akkurat nur die Übersetzung.
Der Wein ist auf dem Markt.

The wine is in the market.

from de to en


In [22]:
print_example(-1)

Please translate the following section into German. Answer only accurately with the translation.
The section is called: The wine is in the market.

Der Wein ist auf dem Markt.

from en to de


# Save Data

In [23]:
!mkdir -p instruct_data

In [24]:
result_df = pd.DataFrame({
    "input": input_texts, 
    "output": output_texts, 
    "src_lang": src_langs, 
    "target_lang": target_langs,
})
result_df["uuid"] = [uuid.uuid4() for _ in range(len(result_df))]
result_df

,input,output,src_lang,target_lang,uuid
0,Gegeben ist folgender Text: Lass uns etwas ver...,Let's try something.,de,en,f809c770-a2dc-447b-9e61-7cfcb6a036e3
1,Translate the following text into German and c...,Lass uns etwas versuchen!,en,de,fbee69e9-9e95-4741-98df-31ffeb14e32b
2,Übersetz den folgenden Text ins Englische und ...,I have to go to sleep.,de,en,a9dcd022-4a25-48f6-a5a2-804409c1bb84
3,Translate the following section into German an...,Ich muss schlafen gehen.,en,de,5d7714b2-1e6f-4176-b4ce-42aa15150bbd
4,"Sag mir, was folgender Text auf Englisch heißt...",What is it?,de,en,156662e8-24b1-4a64-8c45-06a317ccf103
...,...,...,...,...,...
626249,Bitte übersetze folgenden Abschnitt ins Deutsc...,Ihr Vater ist streng.,en,de,e067d944-0ddb-4c74-8fb4-2942b3dd7399
626250,Given is the following section: Der Rotwein ko...,The red wine costs thirty coins.,de,en,1263859f-6b16-402a-a51d-5452b7fb37ac
626251,Given is the following section: The red wine c...,Der Rotwein kostet dreißig Münzen.,en,de,f2a23cbc-bf2f-4596-bf5e-5c3f4be8f69e
626252,"Sag mir, was folgender Text auf Englisch heißt...",The wine is in the market.,de,en,3a9d74d7-7a6c-4ecf-8a9f-d22cd92917ea


In [25]:
result_df.to_csv(f"./instruct_data/{DATASET_NAME_AND_VERSION}.csv.gz", index=False, compression="gzip")